In [2]:
%run package_import.ipynb
%run data_profiling.ipynb
%run data_transformation.ipynb


In [3]:
import os
os.chdir(path='/Users/liuyifu/Project_data')

In [4]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/seattle_airbnb', pool_recycle=3600)
dbConnection    = sqlEngine.connect()


## ELT on calendar csv

In [5]:
def transform_calendar_data(dataframe):
    
    #convert string into numeric values
    dataframe['price'] = transform_prices_column(dataframe,'price')
    #drop null value in the column "price"
    dataframe.dropna(subset=['price'],inplace=True)
    #d
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    
    return dataframe   

In [7]:
df_calendar = pd.read_csv('Seattle_Airbnb/calendar.csv')
df_calendar = transform_calendar_data(df_calendar)
df_calendar.to_sql('raw_calendar', dbConnection, if_exists='replace', index=False)

/var/folders/ll/01gdbc2n2d1f47wbbx616f4r0000gn/T/ipykernel_93802/1151561823.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column] = dataframe[column].str.replace("$", "").str.replace(",", "")


## ELT on Listings csv

In [10]:
def transform_reviews_data(dataframe):
    
    # The majority of the columns below are null values and so they are removed
    dataframe.drop(columns=['license', 'experiences_offered', 'square_feet'],inplace=True)
    
    # As the following columns are relevant to price including '$' and ',' , these charaters need to be deleted.
    # After '$' and ',' are deleted, convert the string into float 64
    price_colums = ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']
    
    for column in price_colums:
        
         # As the following columns are relevant to price including '$' and ',' , these charaters need to be deleted.
         # After '$' and ',' are deleted, convert the string into float 64
         # The formula can be found in the data_transformation.ipynb
        dataframe[column] =  transform_prices_column(dataframe, column)
        # for the relevant pric columns listed, fill zero for those rows with NaN
        dataframe[column] = dataframe[column].fillna(0)
        
    # Define columns relevant to rates with %
    rate_columns = ['host_response_rate', 'host_acceptance_rate']
    
    #As % is included in the rate columns, their data types are string. 
    # Before converting them to numeric, '%' need to be deleted, convert the figure as numeric and then use it to divided by 100
    for column in rate_columns:
        
        # In these columns, skip rows with NaN. 
        # For the rows which are not empty, perform the following exercise to convert the string to numeric
        # 1. replace "#" with ''
        # 2. convert the figure to numeric
        # 3. use the numeric values to be divided by 100 to get the rate which is less than or equal to 1. 
        # For example, 100% is converted to 1; 68% is converted to 0.68. 
        dataframe[column] = dataframe[column].apply(lambda x: pd.to_numeric(x.replace('%',''))/100 if pd.notna(x) else np.nan)

    
    return dataframe

In [17]:
df_listings = pd.read_csv('Seattle_Airbnb/listings.csv')
df_listings =  transform_reviews_data(df_listings)
df_listings.to_sql('raw_listings', dbConnection, if_exists='replace', index=False)

/var/folders/ll/01gdbc2n2d1f47wbbx616f4r0000gn/T/ipykernel_93802/1151561823.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column] = dataframe[column].str.replace("$", "").str.replace(",", "")


### ETL on Reviews csv

In [8]:
df_reviews = pd.read_csv('Seattle_Airbnb/reviews.csv')
df_reviews['date'] = pd.to_datetime(df_reviews['date'])
df_reviews.to_sql('raw_reviews', dbConnection, if_exists='replace', index=False)

In [7]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to everything!
1,7202016,39087409,2015-07-20,32440555,Frank,"Kelly has a great room in a very central location. \r\nBeautiful building , architecture and a s..."
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighborhood. This is the kind of apartment I wish I ha..."
3,7202016,40813543,2015-08-02,33671805,George,"Close to Seattle Center and all it has to offer - ballet, theater, museum, Space Needle, restaur..."
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating in a great neighborhood. She has some great coffee...
